## Before Start

An initial data exploration was done to first gain some insights. The classes look balanced in the train data at about 50% each, and most reviews are of length 100-200. Even after removing stopwords, the most common word (br) is still a stopword since it is just a break in the text, and does not have any meaning. It should be added to the dictionary to be removed.

In [ ]:
import pandas as pd
train_data = pd.read_csv('/Users/kiriharari/Downloads/news_fulltrain.csv', header=None)
train_data.columns=['labels','text']
train_data.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from nltk.tokenize import word_tokenize

# train_data = pd.read_csv('/content/gdrive/MyDrive/news_fulltrain.csv', header=None)
# print(train_data.head())

print("Class Distribution:")
print(train_data['label'].value_counts(normalize=True))

review_lengths = train_data['text'].apply(lambda x: len(word_tokenize(x)))
plt.figure(figsize=(9, 6))
plt.hist(review_lengths, bins=50, alpha=0.7)
plt.title('Distribution of Review Lengths')
plt.xlabel('Reviews Length')
plt.ylabel('Reviews Num')
plt.show()

stop_words = set(stopwords.words('english'))
all_words = [word for review in train_data['text'] for word in word_tokenize(review.lower()) if word.isalpha() and word not in stop_words]

word_freq = Counter(all_words).most_common(20)

words, frequencies = zip(*word_freq)
plt.figure(figsize=(9, 6))
plt.bar(words, frequencies)
plt.title('Top 20 Common Words (Before removing br)')
plt.xticks(rotation=45)
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.show()

stop_words = set(stopwords.words('english'))
stop_words.add('br')
all_words = [word for review in train_data['text'] for word in word_tokenize(review.lower()) if word.isalpha() and word not in stop_words]

word_freq = Counter(all_words).most_common(20)

words, frequencies = zip(*word_freq)
plt.figure(figsize=(10, 6))
plt.bar(words, frequencies)
plt.title('Top 20 Common Words (After removing br)')
plt.xticks(rotation=45)
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.show()


In [ ]:
from sklearn.utils import resample
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

train_data = pd.read_csv('/Users/kiriharari/Desktop/EE6405/BERT_NEWS/news_fulltrain.csv', header=None)
train_data.columns=['labels','text']
train_data = train_data[(train_data['labels'] == 1) | (train_data['labels'] == 4)]
train_data['labels'].unique()

train_data_satire  =train_data[train_data['labels'] == 1]
train_data_reliable  =train_data[train_data['labels'] == 4]

train_data_satire_sampled = resample(train_data_satire,
                                     replace=True,
                                     n_samples=len(train_data_satire),
                                     random_state=42)

train_data_balanced = pd.concat([train_data_satire_sampled, train_data_reliable])
train_data_balanced['labels'] = train_data_balanced['labels'].replace({1:0, 4:1})


test_data = pd.read_csv('/Users/kiriharari/Desktop/EE6405/BERT_NEWS/news_balancedtest.csv')

test_data.columns = ['labels', 'text']
test_data = test_data[(test_data['labels'] == 1) | (test_data['labels'] == 4)]

test_data_balanced = test_data.copy()
test_data_balanced['labels'] = test_data_balanced['labels'].replace({1: 0, 4: 1})
test_data_balanced.head()
train_data_balanced.head()
train_data_balanced = train_data_balanced.reset_index(drop=True)
test_data_balanced = test_data_balanced.reset_index(drop=True)

In [ ]:
import torch
from datasets import Dataset, load_metric
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize(data):
    tokenized_data = tokenizer(data["text"],
                     padding = "max_length",
                     truncation = True,
                     max_length=128,
                     return_tensors='pt')
    tokenized_data['labels'] = torch.tensor(data["labels"])
    return tokenized_data
# tokenized_train = train_data_balanced.map(tokenize, batched=True)
# tokenized_test = test_data_balanced.map(tokenize, batched=True)
config = AutoConfig.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels = 2)

train_dataset = Dataset.from_pandas(train_data_balanced)
test_dataset = Dataset.from_pandas(test_data_balanced)

tokenized_train = train_dataset.map(tokenize, batched=True)
tokenized_test = test_dataset.map(tokenize, batched=True)

training_args = TrainingArguments(
    output_dir="./result",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    f1 = load_metric("f1")
    precision = load_metric("precision")
    recall = load_metric("recall")
    accuracy = load_metric("accuracy")

    f1_score = f1.compute(predictions=predictions, references=labels, average='binary')
    precision_score = precision.compute(predictions=predictions, references=labels, average='binary')
    recall_score = recall.compute(predictions=predictions, references=labels, average='binary')
    accuracy_score = accuracy.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy_score['accuracy'],
        "f1": f1_score['f1'],}
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
# 指定保存模型的路径
model_path = "/Users/kiriharari/Desktop/EE6405/BERT_NEWS/saved_model"

# 保存模型
model.save_pretrained(model_path)

# 同时，保存 tokenizer 到相同的路径
tokenizer.save_pretrained(model_path)


In [ ]:
import torch
from datasets import Dataset, load_metric
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
model_path = "/Users/kiriharari/Desktop/EE6405/BERT_NEWS/saved_model"

# 加载模型
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    f1 = load_metric("f1")
    precision = load_metric("precision")
    recall = load_metric("recall")
    accuracy = load_metric("accuracy")

    f1_score = f1.compute(predictions=predictions, references=labels, average='binary')
    precision_score = precision.compute(predictions=predictions, references=labels, average='binary')
    recall_score = recall.compute(predictions=predictions, references=labels, average='binary')
    accuracy_score = accuracy.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy_score['accuracy'],
        "f1": f1_score['f1'],
        "precision": precision_score['precision'],
        "recall": recall_score['recall']
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)

trainer.evaluate()

In [ ]:
test_string_satire = '''
When so many actors seem content to churn out performances for a quick paycheck, a performer who adheres to his principles really stands out. Thats why Jeff Bridges made waves this week when he announced that from now on, he will only perform nude scenes. In an interview in this months GQ, the Big Lebowski star made it clear that he was more than ready to move on to a new phase in his career, leaving his clothed roles in the past. Ive been there and Ive done that, said Bridges, rattling off a laundry list of the films hes appeared in covered up. Now, I can finally afford to only take on roles that excite me. Right now, those are roles with nude scenes. Why waste my time with anything else? Powerful. Though he made it clear that he doesnt regret his previous non-nude roles, Jeff admitted that hed always struggled with pressure from directors and studios to stay clothed on camera. No more towels; no more bathrobes; no more carefully placed plants, he added. Even if my character isnt written as nude, any director I work with will have to figure out how to make him that way. Itll be a challenge for both of us, and one I cant wait to tackle. For their part, Jeffs fans have been nothing but supportive. Wow! Whether or not you agree with him, youve got to have respect for a Hollywood star with that much conviction. You keep doing you, Jeff!
'''

test_string_reliable = '''
The Alberta province health minister wants to know if swine flu shots were 'inappropriately diverted' to the Calgary Flames while thousands had to stand in line for hours for the vaccine. Alberta Health Minister Ron Liepert says he doesn't know where the NHL team got the vaccine, adding that Alberta Health Services is the only supplier in the province. Team president Ken King says the club contacted the department and asked for the clinic. Health officials have begun an investigation into the special clinic, which was held for the players and their families last Friday. Liepert says the vaccine would be diverted only with the approval of the chief medical officer of health, but he doesn't know if that was the case. Alberta's opposition parties say professional ice hockey players shouldn't be getting the vaccine ahead of cancer patients and pregnant women.
'''
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=1  # 设置为1因为我们只预测一个样本
)
trainer = Trainer(model=model, args=training_args)
print("Satire")
satire_input_x = pd.DataFrame({'labels':[0], 'text': test_string_satire})

reliable_input_x = pd.DataFrame({'labels':[0], 'text': test_string_reliable})

satire_dataset = Dataset.from_pandas(satire_input_x)
reliable_dataset = Dataset.from_pandas(reliable_input_x)

satire_input_x = satire_dataset.map(tokenize, batched=True)
reliable_input_x = reliable_dataset.map(tokenize, batched=True)

# satire_input_x['labels'] = torch.tensor(0)
# Check the number of tokens
# print("Input IDs shape:", satire_input_x['input_ids'].shape)  # 应显示 (1, 128) 或其他类似批处理形状
# print("Input IDs shape:", tokenized_test['input_ids'].shape)
predictions = trainer.predict(satire_input_x)


logits = predictions.predictions
probs = torch.softmax(torch.tensor(logits), dim=-1)
predictions = torch.argmax(probs, dim=-1)

print("Probabilities:", probs)
print("Predictions:", predictions)

print("Reliable")

predictions = trainer.predict(reliable_input_x)


logits = predictions.predictions
probs = torch.softmax(torch.tensor(logits), dim=-1)
predictions = torch.argmax(probs, dim=-1)

print("Probabilities:", probs)
print("Predictions:", predictions)